<a href="https://colab.research.google.com/github/SoumadeepMandal/Smart_chatbot/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string


In [63]:
data_fake = pd.read_csv('/content/Fake.csv')
data_true = pd.read_csv('/content/True.csv')

In [64]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [65]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [66]:
data_fake['class']=0
data_true['class']=1

In [67]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [68]:
data_fake_manual_testing = data_fake.tail(10)
# Correcting the drop method call syntax
for i in range(23480,23470,-1):
   data_fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = data_true.tail(10)
# Correcting the drop method call syntax
for i in range(21416,21406,-1):
   data_true.drop([i], axis = 0, inplace =True)

In [69]:
data_fake.shape,data_true.shape

((23471, 5), (21407, 5))

In [70]:
# Ensure data_fake_manual_testing is a DataFrame by re-assigning from the original data_fake DataFrame
data_fake_manual_testing = data_fake.tail(10)
# Now, assign the 'class' column
data_fake_manual_testing['class'] = 0

# Ensure data_true_manual_testing is a DataFrame by re-assigning from the original data_true DataFrame
data_true_manual_testing = data_true.tail(10)
# Now, assign the 'class' column
data_true_manual_testing['class'] = 1

<ipython-input-70-440f84cc5fad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
<ipython-input-70-440f84cc5fad>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [71]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23461,REPORT: ‘Federal Government Escalated the Viol...,KILLED: Rancher and protest spokesman Robert ...,Middle-east,"January 28, 2016",0
23462,"BOILER ROOM – Oregon Standoff, Cuddle Parties,...",Tune in to the Alternate Current Radio Network...,Middle-east,"January 28, 2016",0
23463,"Eyewitness Says Feds Ambushed Bundys, 100 Shot...",Patrick Henningsen 21st Century Wire UPDATE: 1...,Middle-east,"January 27, 2016",0
23464,Episode #119 – SUNDAY WIRE: ‘You Know the Dril...,Episode #119 of SUNDAY WIRE SHOW finally resum...,Middle-east,"January 24, 2016",0
23465,‘There’ll be boots on the ground’: US making n...,21st Century Wire says Various parties in Wash...,Middle-east,"January 23, 2016",0
23466,Boston Brakes? How to Hack a New Car With Your...,21st Century Wire says For those who still ref...,Middle-east,"January 22, 2016",0
23467,Oregon Governor Says Feds ‘Must Act’ Against P...,"21st Century Wire says So far, after nearly 20...",Middle-east,"January 21, 2016",0
23468,Ron Paul on Burns Oregon Standoff and Jury Nul...,21st Century Wire says If you ve been followin...,Middle-east,"January 21, 2016",0
23469,BOILER ROOM: As the Frogs Slowly Boil – EP #40,Tune in to the Alternate Current Radio Network...,Middle-east,"January 20, 2016",0
23470,Arizona Rancher Protesting in Oregon is Target...,RTOne of the most visible members of the armed...,Middle-east,"January 20, 2016",0


In [72]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21397,Germany's Schulz says he would demand U.S. wit...,BERLIN (Reuters) - The leader of Germany s Soc...,worldnews,"August 23, 2017",1
21398,Blunt instrument? What a list of banned articl...,SHANGHAI (Reuters) - An old review of an acade...,worldnews,"August 23, 2017",1
21399,Saudi police release teenager detained for dan...,DUBAI (Reuters) - A 14-year-old boy who was de...,worldnews,"August 22, 2017",1
21400,"The People's Princess, Britons work to keep me...",LONDON (Reuters) - Abdul Daoud spilt most of t...,worldnews,"August 23, 2017",1
21401,"Argentina labor unions protest job losses, Mac...",BUENOS AIRES (Reuters) - Argentina s main labo...,worldnews,"August 22, 2017",1
21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1
21406,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1


In [73]:
data_merge = pd.concat([data_fake,data_true], axis =0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [74]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [75]:
data=data_merge.drop(['title','subject','date'] , axis =1 )

In [76]:
data.isnull().sum()

,0
text,0
class,0


In [77]:
data=data.sample(frac =1)

In [78]:
data.head()

,text,class
12248,Liberals internal struggle in one clip: .@and...,0
18419,BAGHDAD (Reuters) - Iraq stopped selling dolla...,1
8514,Florida Republicans have sponsored a bill that...,0
4944,WASHINGTON (Reuters) - Leaders of the House of...,1
20910,(Reuters) - The eye of Hurricane Irma was pass...,1


In [79]:
data.reset_index(inplace = True)
data.drop(['index'],axis =1,inplace = True)

In [80]:
data.columns

Index(['text', 'class'], dtype='object')

In [81]:
data.head()

,text,class
0,Liberals internal struggle in one clip: .@and...,0
1,BAGHDAD (Reuters) - Iraq stopped selling dolla...,1
2,Florida Republicans have sponsored a bill that...,0
3,WASHINGTON (Reuters) - Leaders of the House of...,1
4,(Reuters) - The eye of Hurricane Irma was pass...,1


In [82]:
def wordopt(text):
  text= text.lower()
  text=re.sub('\[.*?\]', '' ,text)
  text = re.sub("\\W", " ", text )
  text = re.sub('https?://\S+|www\.S+', '',text)
  text = re.sub('<. *?>+', '',text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '',text)
  text =re.sub('\w*\d\w*', '' ,text)
  return text

In [83]:
data ['text']=data['text'].apply(wordopt)

In [84]:
x=data['text']
y=data['class']

In [85]:
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.25)

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_text)

In [87]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [88]:
pred_lr=LR.predict(xv_test)

In [90]:
LR.score(xv_test, y_test)

0.4996434937611408

In [91]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.53      0.52      0.53      5967
           1       0.47      0.47      0.47      5253

    accuracy                           0.50     11220
   macro avg       0.50      0.50      0.50     11220
weighted avg       0.50      0.50      0.50     11220



In [92]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [93]:
predict_dt=DT.predict(xv_test)


In [95]:
DT.score(xv_test, y_test)

0.4996434937611408

In [96]:
print(classification_report(y_test, predict_dt))

              precision    recall  f1-score   support

           0       0.53      0.52      0.53      5967
           1       0.47      0.47      0.47      5253

    accuracy                           0.50     11220
   macro avg       0.50      0.50      0.50     11220
weighted avg       0.50      0.50      0.50     11220



In [97]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [98]:
predit_gb = GBC.predict(xv_test)

In [99]:
GBC.score(xv_test, y_test)

0.4987522281639929

In [100]:
print(classification_report(y_test, predit_gb))

              precision    recall  f1-score   support

           0       0.53      0.52      0.52      5967
           1       0.47      0.48      0.47      5253

    accuracy                           0.50     11220
   macro avg       0.50      0.50      0.50     11220
weighted avg       0.50      0.50      0.50     11220



In [101]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [102]:
predict_rf = RFC.predict(xv_test)

In [103]:
RFC.score(xv_test,y_test)

0.4987522281639929

In [105]:
print(classification_report(y_test, predict_rf))

              precision    recall  f1-score   support

           0       0.53      0.52      0.53      5967
           1       0.47      0.47      0.47      5253

    accuracy                           0.50     11220
   macro avg       0.50      0.50      0.50     11220
weighted avg       0.50      0.50      0.50     11220



In [106]:
def output_lable(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "True News"


def manual_testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(wordopt)
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(new_xv_test)
  pred_DT = DT.predict(new_xv_test)
  pred_GBC = GBC.predict(new_xv_test)
  pred_RFC = RFC.predict(new_xv_test)

  return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [112]:
news = str(input())
manual_testing(news)

On Sunday, December 4, 2016, a shooting incident occurred at a pizza shop in northwestern Washington D.C. during the middle of the day. This was in a busy but peaceful shopping district, and the shop was filled with families. A man brought a rifle into the shop and began shooting. Fortunately, no one was hurt, and the suspect was arrested, but the motive for this crime and the circumstances that triggered it were shocking.


LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News
